# __Step 4.6: 2D representation of yeast genetic interaction network__

Goals here:
- Use UMAP to plot relationships between:
  - Genes
  - Genes of different pathways, biological processes, molecular functions
  - Documents of the same topics but different time bins

Based on:
- [UMAP on sparse data](https://umap-learn.readthedocs.io/en/latest/sparse.html)

Issues:
- 2/9/23
  - The topic model was generated using older version of BERTopic and it cannot be loaded with the current version.
  - Try instead to get the topic assignment using the probability values.


## ___Set up___

In [1]:
import datatable as dt
import pandas as pd
import numpy as np
import umap
import umap.plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from matplotlib import colors
from pathlib import Path

In [3]:
# Reproducibility
seed = 20230417

# Setting working directory
proj_dir   = Path.home() / "Shiu_Lab/Project"
work_dir   = proj_dir / "Scripts/Data_Vis"

## ___Read data to pre-process___

### Genetic Interaction Score Network

In [ ]:
# pre-process the Costanzo et al. 2016 global genetic interaction network data
costanzo = "/mnt/home/seguraab/Shiu_Lab/Co-function/Data/Costanzo_2016/S1"
EE = dt.fread(f"{costanzo}/SGA_ExE.txt") # essential x essential gene pairs
EN = dt.fread(f"{costanzo}/SGA_ExN_NxE.txt") # essential x nonessential gene pairs
NN = dt.fread(f"{costanzo}/SGA_NxN.txt") # nonessential x nonessential gene pairs

# add interaction type column
EE["Type"] = "ExE"
EN["Type"] = "ExN"
NN["Type"] = "NxN"

# combine and save
data = dt.rbind(EE, EN, NN)
data.to_csv(f"{proj_dir}/Data/Costanzo_2016/SGA_all.csv")
del EE, EN, NN

In [ ]:
# keep only relevant columns
data = dt.fread(f"{proj_dir}/Data/Costanzo_2016/SGA_all.csv")
data = data.to_pandas()
data = data[["Query Strain ID", "Array Strain ID",
             "Genetic interaction score (ε)", "P-value"]]
data.shape # (19313654, 4)

### Genetic Interaction Similarity Network

In [ ]:
gi_sim = dt.fread("~/Shiu_Lab/Project/Data/Costanzo_2016/S3/Global_Similarity_Network.txt")
gi_sim = gi_sim.to_pandas()
gi_sim.head()

### Top predictive genes identified by Random Forest in each environment

In [4]:
## SNPs
rf = pd.read_csv(f"{proj_dir}/Scripts/Data_Vis/SNPs_imp_max_all_sorted.tsv", sep="\t")
rf.set_index("gene", inplace=True)
rf = rf.loc[rf.index!="intergenic"] # drop intergenic SNP category
rf.shape # (5178, 35)

(5178, 35)

## ___Create global genetic interaction adjacency matrix___

### Global Yeast GI Score Network Adjacency Matrix

In [ ]:
# determine which gene pairs interact
# stringent confidence threshold as recommended by 
# Costanzo et al., 2016 (P <0.05 and ε > 0.16 or ε < -0.12)
data["Query Strain ID"] = data["Query Strain ID"].str.split("_").str[0] # keep only gene name
data["Array Strain ID"] = data["Array Strain ID"].str.split("_").str[0]
data = data.loc[data["P-value"] <= 0.05,:] # remove non-significant interactions
data = data.loc[(data["Genetic interaction score (ε)"] >= 0.16) | \
				(data["Genetic interaction score (ε)"] <= -0.12), :]
data.to_csv(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent.csv", index=None) # save
data.shape # (385132, 4)

In [ ]:
# create adjacency matrix
data = pd.read_csv(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent.csv")
data = data.iloc[:,:3]
data.set_index(["Query Strain ID", "Array Strain ID"], inplace=True) # set MultiIndex
data = data[~pd.DataFrame(np.sort(np.array(data.index.tolist()), 1)).duplicated().values]
print(data.shape) # (331853, 1)
adj = pd.pivot_table(data, values="Genetic interaction score (ε)",
					 columns="Array Strain ID", index="Query Strain ID")
print(adj.shape) # (4922, 4450)
adj.fillna(0, inplace=True) # replace NaN (for missing interactions)
print(adj.isna().sum().unique()) # check for missing data
adj.to_csv(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent_adjacency.csv")

### Global Yeast GI Similarity Network Adjacency Matrix

In [ ]:
gi_sim.columns = gi_sim.iloc[0,:]
gi_sim.set_index(gi_sim.iloc[:,1], inplace=True)
gi_sim = gi_sim.iloc[1:,2:]
gi_sim.replace("NaN", "0", inplace=True)
gi_sim = gi_sim.apply(pd.to_numeric, axis=1)
gi_sim.to_csv(f"{proj_dir}/Data/Costanzo_2016/S3/Global_Similarity_Network_adjacency.csv")
gi_sim.head()

### Environment-specific GI Network adjacency Matrices [DON'T NEED TO DO THIS]

In [ ]:
# get genetic interactions of top predictive genes in each environment
# gi_rf = pd.DataFrame(columns=["Query Strain ID", "Array Strain ID", "Genetic interaction score (ε)", "Env"])
# gi_rf = []
# not_in_query = []
# for env in rf.columns: # loop through all environments
# 	genes = rf[env].loc[~rf[env].isna()].index # subset top genes in env
# 	for gene1 in genes:
# 		if gene1 in data.index.get_level_values("Query Strain ID"):
# 			# get genetic interaction information of top genes in env based on Query gene
# 			gi_gene1 = data.iloc[data.index.get_level_values("Query Strain ID")==gene1]
# 			gi_gene1["Env"] = env # add environment label
# 			gi_gene1.reset_index(inplace=True)
# 			# gi_rf = pd.concat([gi_rf, gi_gene1], axis=0) # combine with gi_rf
# 			gi_rf.append(gi_gene1)
# 		# else:
# 			# not_in_query.append(gene1) # gene has no interactions'
# gi_rf = pd.concat(gi_rf, axis=0)
# print(gi_rf.shape) # (1381788, 4)
# gi_rf.to_csv(
# 	f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent_adjacency_only_envs.csv",
# 	index=None, chunksize=1000)
# print(len(set(not_in_query))) # 1071 unique genes
# with open(f"{proj_dir}/Data/Costanzo_2016/top_genes_any_env_not_in_SGA_all_stringent.txt", "w") as f:
# 	for gene in not_in_query:
# 		f.write(f"{gene}\n")
# not_in_query = set(not_in_query)

In [ ]:
# read in combined adjacency matrix for all envs
# gi_rf = dt.fread(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent_adjacency_only_envs.csv")
# gi_rf = gi_rf.to_pandas()

In [ ]:
# make adjacency matrices for each environment
# print(len(gi_rf.Env.unique())) # 35 environments
# for env in gi_rf.Env.unique():
# 	env_gi = gi_rf.loc[gi_rf.Env==env]
# 	adj_env = pd.pivot_table(env_gi, values="Genetic interaction score (ε)",
# 							 columns="Array Strain ID", index="Query Strain ID")
# 	adj_env.fillna(0, inplace=True)
# 	adj_env.to_csv(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent_adjacency_{env}.csv")

## ___UMAP run___

### Functions

In [4]:
# make genes array
def make_genes_array(global_genes, env_genes):
    """Make a binary topics_array with 1,0 values, which tells umap which
    points to color
    Args:
        global_genes (list): query genes in yeast global genetic interaction network
            env
        env_genes (list): query genes of a specific environment
    """
    genes_array = []
    for gene in global_genes:
        if gene in env_genes:
            genes_array.append(1),
        else:
            genes_array.append(0)
    genes_array = np.array(genes_array)
    return genes_array

In [18]:
def get_mapper(embeddings, mapper_dir,
	file="mapper_topics_all.pickle", n_neighbors=15, min_dist=0.1,
	metric='cosine',random_state=seed):
	''' Read or generate UMAP fitted obj
	Args:
		mapper_dir (Posix path) - location where mapper file is or should be saved
		n_neighbors (int) - UMAP parameter, for constraining the size of the local 
			neighborhood UMAP will look at when attempting to learn the manifold 
			structure of the data
		min_dist (float) - UMAP parameter, minimum distance apart that points are 
			allowed to be in the low dimensional representation
	Return:
		mapper
	'''
	# Create dir if it does not exist
	mapper_dir.mkdir(parents=True, exist_ok=True)
	mapper_file = mapper_dir / file
	# read if exists
	if mapper_file.is_file():
		with open(mapper_file, "rb") as f:
			mapper = pickle.load(f)
	# else create mapper obj and save
	else:
		model  = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,
						   metric=metric, random_state=random_state)
		mapper = model.fit(embeddings)
		with open(mapper_file, "wb") as f:
			pickle.dump(mapper, f)
	return mapper

In [5]:
# get colors
#To get color bins:
#https://stackoverflow.com/questions/69085926/have-each-histogram-bin-with-a-different-color
#Matplotlib color map
#https://matplotlib.org/stable/gallery/color/colormap_reference.html 
#RGB and RGBA
#https://matplotlib.org/stable/tutorials/colors/colors.html
# rgb or rgba won't work for color_keys later, need hex color
#https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.colors.to_hex.html#matplotlib.colors.to_hex
def get_ckeys(topics_array):
	cm = plt.cm.rainbow
	ckeys = {i:colors.to_hex(cm(i/len(set(topics_array)))) for i in range(len(set(topics_array)))}
	return ckeys

In [6]:
# use default color map
#umap.plot.points(mapper, labels=topics_array)
#mapper_topics_all_plot = work_dir / 'fig_4_6_mapper_topics_all.pdf'

def plot_overall(mapper_dir, mapper, labels, color_key, topic):
	umap.plot.points(mapper, labels=labels, color_key=color_key)
	mapper_topics_all_plot = mapper_dir / f'fig_4_6_mapper_topics_all_{topic}_labelled.pdf'
	plt.savefig(mapper_topics_all_plot)

In [7]:
def umap_plot_each(mapper_dir, mapper, labels, topic, color):
	color_key={f"topic={topic}":color, "all_others":"lightgray"}
	umap.plot.points(mapper, labels=labels, color_key=color_key)
	mapper_topic_plot = mapper_dir / f'fig_4_6_mapper_topic_{topic}.pdf'
	plt.title(f'Topic {topic}')
	plt.savefig(mapper_topic_plot)
	plt.close()

In [8]:
# go through topics
def plot_each(mapper_dir, mapper, topics_array, ckeys): #, env):
	for topic in range(-1, len(topics_array)):
		topic_labels = []
		color = ckeys[topic+1]

		# Modify labels
		for label in topics_array:
			if label == topic:
				topic_labels.append(f"topic={topic}")
			else:
				topic_labels.append("all_others")

		# Change label list into an array for umap.plot
		labels_array =  np.array(topic_labels)

		# Change legend labels (haven't tested yet)
		# if topic==1:
		# 	topic = env
		# elif topic==0:
		# 	topic = "Yeast Global GI Network"
		# Plotting
		umap_plot_each(mapper_dir, mapper, labels_array, topic, color)

### __Default parameters for Yeast GI Score Network Adjacency Matrix__

In [9]:
mapper_dir = work_dir / '_umap_default'
global_embeddings = dt.fread(f"{proj_dir}/Data/Costanzo_2016/SGA_all_stringent_adjacency.csv", )
global_embeddings = global_embeddings.to_pandas()
global_embeddings.set_index("Query Strain ID", inplace=True)
global_genes = global_embeddings.index.to_list()
global_embeddings.head()

,YAL001C,YAL002W,YAL004W,YAL005C,YAL007C,YAL008W,YAL010C,YAL011W,YAL013W,YAL014C,...,YPR192W,YPR193C,YPR194C,YPR195C,YPR196W,YPR197C,YPR198W,YPR199C,YPR200C,YPR201W
Query Strain ID,,,,,,,,,,,,,,,,,,,,,
YAL001C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL007C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL008W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL009W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### UMAP of Global Yeast GI Score Network Pathway Modules

In [ ]:
# Pathway annotations
map = pd.read_csv(f"~/Shiu_Lab/Co-function/Data/MetaCyc/All-genes-pathways-S288c_pivoted.txt")
# descriptions = pd.read_csv(f"~/Shiu_Lab/Co-function/Data/MetaCyc/All-pathways-S288c_descriptions.txt", sep="\t")
map.head() #, descriptions.head()

In [ ]:
map = map[["Accession.1", "Pathways.of.gene"]]
map.fillna("No_annotation", inplace=True)
map.drop_duplicates(inplace=True)
map.set_index("Accession.1", inplace=True)
map.to_csv(f"{proj_dir}/Data/Costanzo_2016/S288C_pathways.csv")
map.shape
map.head()

In [10]:
# assign pathway labels to each gene
# Note: some genes have multiple pathways. what to do?
map = pd.read_csv(f"{proj_dir}/Data/Costanzo_2016/S288C_pathways.csv", index_col=0)
pwys_array = []
for gene in global_genes:
    if gene in map.index:
        if map.loc[gene].shape[0] > 1:
            pwys_array.append("Multiple_pwys")
        else:
            pwys_array.append(map.loc[gene].values[0]) # only the first pathway annotation
    else:
        pwys_array.append("No_annotation")
print(len(pwys_array)) # 4922 (matches row number of global_embeddings)
print(pwys_array)

4922
['No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'Multiple_pwys', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'PWY-7921', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'Multiple_pwys', 'PWY-8145', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'Multiple_pwys', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'Multiple_pwys', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'GLUTSYNIII-PWY', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 'No_annotation', 

In [11]:
# assign a unique number to each pathway
pwys_array2 = {pwy: index for index, pwy in enumerate(set(pwys_array))}
print(pwys_array2)

{'PWY-5760': 0, 'PWY3O-123': 1, 'PWY-5516': 2, 'MANNOSYL-CHITO-DOLICHOL-BIOSYNTHESIS': 3, 'Multiple_pwys': 4, 'PYRUVDEHYD-PWY': 5, 'ARG-PRO-PWY': 6, 'PWY-6610': 7, 'PWY-7918': 8, 'SPHINGOLIPID-SYN-PWY': 9, 'PWY-6773': 10, 'PWY-7250': 11, 'PWY-6012-1': 12, 'GLUTAMATE-DEG1-PWY': 13, 'PWY-5386-1': 14, 'TRNA-CHARGING-PWY': 15, 'PWY-6168': 16, 'GLUCONSUPER-PWY': 17, 'PWY-6756': 18, 'PWY-7036': 19, 'PWY0-1535': 20, 'PWY-2301': 21, 'PWY-6689': 22, 'PWY0-1507': 23, 'PWY-5026': 24, 'LYSINE-AMINOAD-PWY': 25, 'PWY-6829': 26, 'PWY-6825': 27, 'PWY-6842-1': 28, 'PWY3O-4': 29, 'THIOREDOX-PWY': 30, 'PWY-4361-1': 31, 'PWY-7888': 32, 'PWY-46': 33, 'PWY66-429': 34, 'PWY-5277': 35, 'PWY-6981': 36, 'PWY-7282': 37, 'PWY-7424': 38, 'PROUT-PWY': 39, 'HEXPPSYN-PWY': 40, 'PWY-7184': 41, 'PROSYN-PWY': 42, 'PWY-7921': 43, 'TRIGLSYN-PWY': 44, 'PWY-5067': 45, 'PWY-5966-1': 46, 'PWY-4621': 47, 'PWY3O-48': 48, 'PWY-7891': 49, 'PWY66-423': 50, 'PERIPLASMA-NAD-DEGRADATION': 51, 'PWY-7910': 52, 'PWY-6619': 53, 'PWY-6938

In [12]:
# assign each gene to a pathway-specific number
pwys_array3 = []
for pwy in pwys_array:
    pwys_array3.append(pwys_array2[pwy])
print(pwys_array3)

[71, 71, 71, 71, 71, 71, 71, 4, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 43, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 4, 65, 71, 71, 71, 71, 4, 71, 71, 71, 71, 71, 71, 4, 71, 71, 71, 71, 71, 107, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 29, 71, 71, 71, 71, 104, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 4, 71, 71, 71, 71, 71, 71, 71, 26, 71, 71, 71, 71, 71, 71, 71, 71, 4, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 84, 71, 71, 114, 71, 71, 71, 15, 71, 71, 4, 71, 3, 64, 71, 71, 71, 71, 71, 71, 95, 71, 71, 71, 4, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 4, 94, 40, 71, 87, 71, 71, 71, 71, 4, 89, 71, 71, 63, 4, 63, 22, 36, 71, 71, 78, 71, 4, 71, 71, 71, 71, 4, 9, 71, 36, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 9, 71, 71, 26, 71, 71, 71, 71, 71, 71, 71, 3, 71, 71, 71, 71, 71, 71, 71, 64, 71, 71, 4, 71, 71, 71, 71, 71, 71, 71, 4, 71, 71, 71, 4, 71, 71, 71, 113,

In [ ]:
pwys_array2["No_annotation"], pwys_array2["Multiple_pwys"]

In [19]:
# global_embeddings = global_embeddings.to_numpy()
mapper_def = get_mapper(embeddings=global_embeddings, mapper_dir=mapper_dir,
                        save="mapper_costanzo_gi_score_stringent.pickle", n_neighbors=15,
                        min_dist=0.1, metric='cosine', random_state=seed)

: 

: 

In [15]:
ckeys = get_ckeys(pwys_array2.values())
ckeys[pwys_array2["No_annotation"]] = "#d3d3d3" # set No annotation to gray
ckeys[pwys_array2["Multiple_pwys"]] = "#d3d3d3"

In [ ]:
plot_overall(mapper_dir, mapper_def, np.array(pwys_array3), ckeys, "pathways")

#### Run on one environment

In [ ]:
# use one environment to debug code
env = "YPDCAFEIN40"
env_embeddings = rf[env].loc[~rf[env].isna()] # exclude NaNs
env_embeddings_sub = env_embeddings.sort_values(ascending=False).iloc[:200] # top 200 genes
env_embeddings_sub.head(), env_embeddings_sub.shape

In [ ]:
topics_array = make_topics_array(global_embeddings, env_embeddings_sub)
topics_array.sum(), len(topics_array) # env_embeddings genes as 1, all other genes as 0, not all genes may be in the global adj mat

In [ ]:
global_embeddings = global_embeddings.to_numpy()
mapper_def = get_mapper(embeddings=global_embeddings, mapper_dir=mapper_dir,
                        file="mapper_topics_all.pickle", n_neighbors=15,
                        min_dist=0.1, metric='cosine', random_state=seed)
mapper_def

In [ ]:
ckeys = get_ckeys(topics_array)
ckeys

In [ ]:
ckeys = {1: '#8000ff', 0: '#80ffb4'}
plot_overall(mapper_dir, mapper_def, topics_array, ckeys, env)

#### Run UMAP on all the environments

In [ ]:
global_embeddings = global_embeddings.to_numpy()
mapper_def = get_mapper(embeddings=global_embeddings, mapper_dir=mapper_dir,
                        file="mapper_topics_all.pickle", n_neighbors=15,
                        min_dist=0.1, metric='cosine', random_state=seed)

for env in rf.columns:
    env_embeddings = rf[env].loc[~rf[env].isna()] # exclude NaNs
    env_embeddings_sub = env_embeddings.sort_values(ascending=False).iloc[:200] # top 200 genes
    env_genes = env_embeddings_sub.index.to_list()
    genes_array = make_topics_array(global_genes, env_genes) # gene mapping
    # ckeys = get_ckeys(genes_array)
    ckeys = {1: '#8000ff', 0: '#80ffb4'}
    plot_overall(mapper_dir, mapper_def, genes_array, ckeys, env + "_top200")


In [ ]:
for env in rf.columns:
    env_embeddings = rf[env].loc[~rf[env].isna()] # exclude NaNs
    env_embeddings_sub = env_embeddings.sort_values(ascending=False).iloc[:50] # top 50 genes
    env_genes = env_embeddings_sub.index.to_list()
    genes_array = make_topics_array(global_genes, env_genes) # gene mapping
    # ckeys = get_ckeys(genes_array)
    ckeys = {1: '#8000ff', 0: '#80ffb4'}
    plot_overall(mapper_dir, mapper_def, genes_array, ckeys, env + "_top50")

### __Default parameters for Yeast GI Similarity Network Adjacency Matrix__

In [ ]:
mapper_dir = work_dir / '_umap_default'
global_sim_embeddings = dt.fread(f"{proj_dir}/Data/Costanzo_2016/S3/Global_Similarity_Network_adjacency.csv")
global_sim_embeddings = global_sim_embeddings.to_pandas()
global_sim_embeddings.set_index("C0", inplace=True)
global_sim_genes = global_sim_embeddings.index.to_list()
global_sim_embeddings.head()

#### UMAP of Global Yeast GI Similarity Network Pathway Modules

In [ ]:
map = pd.read_csv(f"{proj_dir}/Data/Costanzo_2016/S288C_pathways.csv", index_col=0)
map.head()

In [ ]:
pwys_array = []
for gene in global_sim_genes:
    if gene in map.index:
        pwys_array.append(map.loc[gene].values[0][0]) # only the first pathway annotation
    else:
        pwys_array.append("No annotation")
len(pwys_array) # 5615

In [ ]:
pwys_array2 = {ni: indi for indi, ni in enumerate(set(pwys_array))}
numbers = [pwys_array2[ni] for ni in pwys_array2]
print(pwys_array2)
print(numbers)

In [ ]:
pwys_array3 = []
for pwy in pwys_array:
    pwys_array3.append(pwys_array2[pwy])
print(pwys_array3)

In [ ]:
global_sim_embeddings = global_sim_embeddings.to_numpy()
mapper_sim_def = get_mapper(embeddings=global_sim_embeddings, mapper_dir=mapper_dir,\
                        file="mapper_global_gi_sim.pickle", n_neighbors=15,\
                        min_dist=0.1, metric='cosine', random_state=seed)
ckeys = get_ckeys(numbers)
ckeys[pwys_array2["No annotation"]] = "#d3d3d3" # set No annotation to gray

In [ ]:
plot_overall(mapper_dir, mapper_sim_def, np.array(pwys_array3), ckeys, "sim_pathways")

## ___Testing___

https://umap-learn.readthedocs.io/en/latest/sparse.html

In [ ]:
primes = list(sympy.primerange(2, 110000))
prime_to_column = {p:i for i, p in enumerate(primes)}

In [ ]:
lil_matrix_rows = []
lil_matrix_data = []
for n in range(100000):
    prime_factors = sympy.primefactors(n)
    lil_matrix_rows.append([prime_to_column[p] for p in prime_factors])
    lil_matrix_data.append([1] * len(prime_factors))
len(lil_matrix_rows), len(lil_matrix_data)

In [ ]:
factor_matrix = scipy.sparse.lil_matrix((len(lil_matrix_rows), len(primes)), 
                                        dtype=np.float32)

In [ ]:
factor_matrix.rows = np.array(lil_matrix_rows)
factor_matrix.data = np.array(lil_matrix_data)

In [ ]:
factor_matrix.shape

In [ ]:
mapper = umap.UMAP(metric='cosine', random_state=42, low_memory=True).fit(
                                                                factor_matrix)

In [ ]:
#umap.plot.points(mapper, values=np.arange(100000), theme='viridis')
umap.plot.points(mapper, values=np.arange(100000))

### Testing drawing each topic

In [ ]:
topic_labels = []
topic = -1
for label in topics_array:
  if label == topic:
    topic_labels.append(f"topic={topic}")
  else:
    topic_labels.append("all_others")

labels_array =  np.array(topic_labels)

umap_plot_each(mapper, labels_array, topic)


### Color map

In [ ]:
cmap = plt.cm.viridis
for i in range(0,10):
  print(cmap(1/10))

In [ ]:
decades = np.arange(1910, 2020, 10)
data = np.random.gamma(4, scale=0.2, size=1000)*110+1910

fig, ax = plt.subplots(figsize=(8,4), facecolor='w')
cnts, values, bars = ax.hist(data, edgecolor='k', bins=decades)
ax.set_xticks(decades)
cmap = plt.cm.viridis

for i, (cnt, value, bar) in enumerate(zip(cnts, values, bars)):
    bar.set_facecolor(cmap(cnt/cnts.max()))

### umap.plot.points

In [ ]:
help(umap.plot.points)

In [ ]:
ckeys = {-1:gray}
for topic, rgba in enumerate(rgbas):
  # background is set to (0,0,0) so need need to think about it
  #rgb_fore = rgba[:3]
  #alpha    = rgba[3]
  #rgb      = [rgb_f*alpha for rgb_f in rgb_fore]
  ckeys[topic] = rgba